In [ ]:
from fenics import *
import sympy as sym
from sympy import symbols, pi
from sympy import derive_by_array, tensorcontraction

expr_degree = 2

material_parameter = dict()
material_parameter["c"] = 1
material_parameter["kappa"] = 1
material_parameter["lmbda"] = 1e12
material_parameter["mu_s"] = 1
material_parameter["rho_s"] = 1
material_parameter["mu_f"] = 1
material_parameter["rho_f"] = 1
material_parameter["alpha"] = 1.0

c, kappa, lmbda, mu_s, rho_s, mu_f,rho_f, alpha, t = sym.symbols('c, kappa, lambda, mu_s, rho_s, mu_f,rho_f, alpha, t')

x,y,z = sym.symbols("x, y, z")

dx = (sym.sin(4*pi*x) + sym.sin(3*pi*y))
dy = (sym.cos(2*pi*x) + sym.sin(5*pi*y))

d = sym.Matrix([dx, dy, 0])*t
pP = (sym.cos(3*pi*x) + sym.sin(1*pi*y))*t

def row_wise_div(tensor):
    return sym.Matrix(tensorcontraction(derive_by_array( tensor, [x,y,z]), (0,2)))
    
def row_wise_grad(u):
    return u.jacobian([x,y,z])

def eps(u):
    return 0.5*(row_wise_grad(u) + sym.transpose(row_wise_grad(u)))

def div(u):
    return sym.trace(u.jacobian([x,y,z]))

def grad(p):
    return sym.Matrix([sym.diff(p,x), sym.diff(p,y), sym.diff(p,z)])


n = [1,0,0]
tang = [0,1,0]

phi = alpha*pP - lmbda*div(d) 
f_biot = - row_wise_div( 2*mu_s*eps(d) + phi*sym.eye(3) )
g =  (c + (alpha**2)/lmbda)*sym.diff(pP, t) - alpha/lmbda*sym.diff(phi, t) - div(kappa/mu_f*grad(pP))

ux = (sym.diff(d, t) - kappa/mu_f *grad(pP))[0]
#pF = 2*mu_f*eps(u) - (2*mu_s*eps(d) + phi*sym.eye(3))

#pF.simplify()

In [ ]:
t

In [ ]:


f.simplify()
g.simplify()

p_N_right = - sym.diff( pP, x).subs(x, 1)
p_N_left = sym.diff(pP, x).subs(x, 0)
p_N_bottom = sym.diff(pP, y).subs(y, 0)
p_N_top = - sym.diff(pP, y).subs(y, 1)


fx, fy = f[0], f[1]
dx, dy = u[0], u[1]

xcpp, ycpp = sym.symbols('x[0], x[1]')

variables_scalar = [g, pP, p_N_right, p_N_left, p_N_bottom, p_N_top]
variables_scalar_cpp  = [sym.printing.ccode(v.subs(x, xcpp).subs(y, ycpp)) for v in variables_scalar]
variables_scalar_expr  = [Expression(v, **material_parameter, t=0, degree=expr_degree)
                          for v in variables_scalar_cpp]
g, p_e, p_N_right, p_N_left, p_N_bottom, p_N_top = variables_scalar_expr


variables_vector = [f, u]
variables_vector_cpp  = [(sym.printing.ccode(v[0].subs(x, xcpp).subs(y, ycpp)),
                          sym.printing.ccode(v[1].subs(x, xcpp).subs(y, ycpp)))
                          for v in variables_vector]
variables_vector_expr  = [Expression(v, **material_parameter, t=0, degree=expr_degree)
                          for v in variables_vector_cpp]
f, u_e = variables_vector_expr

In [ ]:
N = 20
p_e.t = 0.2
mesh = UnitSquareMesh(N, N)
plot(p_e, mesh=mesh)

In [ ]:
cd ~/Documents/brain-force-inversion/brain-inversion/

In [ ]:
from fenics import *
from multiphenics import *
import numpy as np
from braininversion.IOHandling import (read_mesh_from_h5, write_to_xdmf, 
                                       xdmf_to_unstructuredGrid, read_xdmf_timeseries)
from braininversion.PlottingHelper import (plot_pressures_and_forces_timeslice, 
                                           plot_pressures_and_forces_cross_section,
                                           extract_cross_section, style_dict)
import matplotlib.pyplot as plt
import yaml
import pyvista as pv
from pathlib import Path
from braininversion.PostProcessing import (get_source_expression,
                                           scalar_bars,
                                           load_meshes)
import os
import ufl
import sys

mesh_name = "MRIExampleSegmentation_Nvcoarse"
sim_name = "baladontBrainSim"

sim_config_file = f"results/{mesh_name}_{sim_name}/config.yml"
mesh_config_file = f"meshes/{mesh_name}/{mesh_name}_config.yml"
plot_dir = f"results/{mesh_name}_{sim_name}/plots/"
try:
    os.mkdir(plot_dir)
except FileExistsError:
    pass
porous_id = 1
fluid_id = 2
interface_id = 1
spinal_outlet_id = 3
mmHg2Pa = 132.32
m3tomL = 1e6

with open(mesh_config_file) as conf_file:
    mesh_config = yaml.load(conf_file, Loader=yaml.FullLoader)
with open(sim_config_file) as conf_file:
    sim_config = yaml.load(conf_file, Loader=yaml.FullLoader)  

mesh, subdomain_marker, label_marker, boundary_marker, label_boundary_marker = load_meshes(mesh_name)
gdim = mesh.geometric_dimension()
sim_file = f"results/{mesh_name}_{sim_name}/results.xdmf"
source_conf = sim_config["source_data"]
T = sim_config["T"]
num_steps = sim_config["num_steps"]
dt = T/num_steps
times = np.linspace(0, T, num_steps + 1)
source_expr = get_source_expression(source_conf, mesh, subdomain_marker, porous_id, times)
probes = mesh_config["probes"]
flatprobes = dict(**probes["sas"],**probes["parenchyma"],**probes["ventricular_system"])
domains = mesh_config["domains"]
name_to_label = {dom["name"]:dom["id"] for dom in domains}


In [ ]:
plotname = "source_inflow"
source_inflow = []
for i in range(num_steps + 1):
    source_expr.i = i
    source_inflow.append(source_expr(Point([0,0,0])))
source_inflow = np.array(source_inflow)
dx_par = Measure("dx", domain=mesh, subdomain_data=label_marker, subdomain_id = name_to_label["parenchyma"])
parenchyma_vol = assemble(Constant(1)*dx_par)
plt.figure(figsize=(10,8))
plt.plot(times, source_inflow*parenchyma_vol*m3tomL, "-*")
plt.grid()
plt.xlabel("t in s")
plt.ylabel("inflow [ml/s]")
plt.title("net blood inflow")

In [ ]:
source_expr.i = 220
source_expr(Point(0,0,0))